In [1]:
import glob
import time
import os
import pandas as pd
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
from argparse import ArgumentParser, Namespace
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from itertools import chain
from tqdm import tqdm
import copy
import shutil
import pickle

In [2]:
es_url = 'http://ckg07:9200'
es_index = 'wikidatadwd-augmented'

# Input Paths

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-canonical/
# train_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/train1-canonical"
train_path = "/Users/amandeep/Github/table-linker/data/SemTabR4_T2DV2_V2/train1-canonical"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-canonical/
# dev_path = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/dev-canonical"
dev_path = "/Users/amandeep/Github/table-linker/data/SemTabR4_T2DV2_V2/dev-canonical/"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/ground_truth/Xinting_GT_csv
# ground_truth_files = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/round4_gt_with_labels"
ground_truth_files = "/Users/amandeep/Github/table-linker/data/SemTabR4_T2DV2_V2/gt"


# OUTPUT PATHS
output_path = "/Users/amandeep/Github/table-linker/data/SemTabR4_T2DV2_V2/table-linker"
train_output_path = f'{output_path}/train1-output/pseudo-gt-nn'
dev_output_path = f'{output_path}/dev-output/pseudo-gt-nn'

# increase version to create a new folder for an experiment
PREVIOUS_VERSION = "v18"
VERSION = "v19"

train_candidate_path = f'{train_output_path}/{VERSION}/candidates'
train_string_feature_path = f'{train_output_path}/{VERSION}/string_features'
train_feature_path = f'{train_output_path}/{VERSION}/features'
train_context_feature_path = f'{train_output_path}/{VERSION}/context_features'
train_missing_candidates_path = f'{train_output_path}/{VERSION}/train_missing_candidates_path'

dev_candidate_path = f'{dev_output_path}/{VERSION}/candidates'
dev_feature_path = f'{dev_output_path}/{VERSION}/features'
dev_string_feature_path = f'{dev_output_path}/{VERSION}/string_features'
dev_context_feature_path = f'{dev_output_path}/{VERSION}/context_features'
dev_output_predictions = f'{dev_output_path}/{VERSION}/dev_predictions'
dev_predictions_top_k = f'{dev_output_path}/{VERSION}/dev_predictions_top_k'
dev_colorized_path = f'{dev_output_path}/{VERSION}/dev_predictions_colorized'
dev_metrics_path = f'{dev_output_path}/{VERSION}/dev_predictions_metrics'
dev_missing_candidates_path = f'{dev_output_path}/{VERSION}/dev_missing_candidates_path'

aux_field = 'graph_embedding_complex,class_count,property_count,context'


train_prop_count = f'{train_output_path}/{VERSION}/train_prop_count' 
train_class_count = f'{train_output_path}/{VERSION}/train_class_count'
train_context_path = f'{train_output_path}/{VERSION}/train_context'
train_graph_embedding = f'{train_output_path}/{VERSION}/train_graph_embedding'

dev_prop_count = f'{dev_output_path}/{VERSION}/dev_prop_count'
dev_class_count = f'{dev_output_path}/{VERSION}/dev_class_count'
dev_context_path = f'{dev_output_path}/{VERSION}/dev_context'
dev_graph_embedding = f'{dev_output_path}/{VERSION}/dev_graph_embedding'

temp_dir = f'{output_path}/temp'

tl_log_file =f'{temp_dir}/tl_log.txt'

pos_output = f'{temp_dir}/training_data/pos_features.pkl'
neg_output = f'{temp_dir}/training_data/neg_features.pkl'
min_max_scaler_path = f'{temp_dir}/training_data/normalization_factor.pkl'

final_score_column = 'siamese_prediction'
threshold = final_score_column+":median"

model_save_path = f'{dev_output_path}/{VERSION}/saved_models'
best_model_path = ''

copy_candidates_from_previous_version = False
copy_string_features_from_previous_version = False

In [3]:
!mkdir -p "$temp_dir"

!mkdir -p "$train_prop_count"
!mkdir -p "$dev_prop_count"
!mkdir -p "$train_class_count"
!mkdir -p "$dev_class_count"
!mkdir -p "$train_graph_embedding"
!mkdir -p "$dev_graph_embedding"
!mkdir -p "$train_context_path"
!mkdir -p "$dev_context_path"

!mkdir -p "$train_candidate_path"
!mkdir -p "$dev_candidate_path"

!mkdir -p "$train_feature_path"
!mkdir -p "$train_string_feature_path"
!mkdir -p "$train_context_feature_path"
!mkdir -p "$dev_feature_path"
!mkdir -p "$dev_string_feature_path"
!mkdir -p "$dev_context_feature_path"

!mkdir -p "$temp_dir/training_data"
!mkdir -p "$dev_output_predictions"
!mkdir -p "$model_save_path"
!mkdir -p "$dev_predictions_top_k"
!mkdir -p "$dev_colorized_path"
!mkdir -p "$dev_metrics_path"
!mkdir -p "$dev_missing_candidates_path"
!mkdir -p "$train_missing_candidates_path"

In [ ]:
if copy_candidates_from_previous_version:
    !cp $dev_output_path/$PREVIOUS_VERSION/candidates/*csv $dev_output_path/$VERSION/candidates
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_prop_count/* $dev_output_path/$VERSION/dev_prop_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_class_count/* $dev_output_path/$VERSION/dev_class_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_context/* $dev_output_path/$VERSION/dev_context
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_graph_embedding/* $dev_output_path/$VERSION/dev_graph_embedding
    
    !cp $train_output_path/$PREVIOUS_VERSION/candidates/*csv $train_output_path/$VERSION/candidates
    !cp $train_output_path/$PREVIOUS_VERSION/train_prop_count/* $train_output_path/$VERSION/train_prop_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_class_count/* $train_output_path/$VERSION/train_class_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_context/* $train_output_path/$VERSION/train_context
    !cp $train_output_path/$PREVIOUS_VERSION/train_graph_embedding/* $train_output_path/$VERSION/train_graph_embedding

In [ ]:
features = ["monge_elkan","monge_elkan_aliases","jaro_winkler","levenshtein","singleton","pgr_rts","context_score","smc_class_score","smc_property_score"]

## Candidate Generation

In [ ]:
def candidate_generation(path, gt_path, output_path, class_count_path, prop_count_path, context_path, graph_embedding):
    file_list = glob.glob(path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        st = time.time()
        filename = file.split('/')[-1]
        gt_file = f"{ground_truth_files}/{filename}"
        output_file = f"{output_path}/{filename}"
        
        !tl --log-file $tl_log_file clean -c label -o label_clean "$file" / \
        --url $es_url --index $es_index \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        --url $es_url --index $es_index \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        / get-ngram-matches -c label_clean  \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" \
        / ground-truth-labeler --gt-file "$gt_file" > "$output_file"

        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = f"{class_count_path}/{filename.strip('.csv')}_class_count.tsv"
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = f"{prop_count_path}/{filename.strip('.csv')}_prop_count.tsv"
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            elif field == 'context':
                context_file = f"{context_path}/{filename.strip('.csv')}_context.tsv"
                aux_df.to_csv(context_file, sep='\t', index=False)
            else:
                graph_embedding_file = f"{graph_embedding}/{filename.strip('.csv')}_graph_embedding_complex.tsv"
                aux_df.to_csv(graph_embedding_file, sep='\t', index=False)
        

In [ ]:
if not copy_candidates_from_previous_version:
    candidate_generation(train_path, ground_truth_files, train_candidate_path, train_class_count, train_prop_count, train_context_path,train_graph_embedding)

In [ ]:
if not copy_candidates_from_previous_version:
    candidate_generation(dev_path, ground_truth_files, dev_candidate_path, dev_class_count, dev_prop_count, dev_context_path, dev_graph_embedding)

## Feature Generation

In [4]:
def string_feature_generation(candidate_dir, output_path):
    file_list = glob.glob(candidate_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]
        output_file = f"{output_path}/{filename}"
        !tl --log-file $tl_log_file string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan --threshold 0.85 $file \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases --threshold 0.85  \
            / string-similarity -i --method jaro_winkler -o jaro_winkler --threshold 0.85  \
            / string-similarity -i --method levenshtein -o levenshtein --threshold 0.85  \
            / create-singleton-feature -o singleton \
            > $output_file

In [5]:
if not copy_string_features_from_previous_version:
    string_feature_generation(train_candidate_path, train_string_feature_path)
else:
    !cp $dev_output_path/$PREVIOUS_VERSION/string_features/*csv $dev_output_path/$VERSION/string_features

345it [2:00:13, 20.91s/it]


In [6]:
if not copy_string_features_from_previous_version:
    string_feature_generation(dev_candidate_path, dev_string_feature_path)
else:
    !cp $train_output_path/$PREVIOUS_VERSION/string_features/*csv $train_output_path/$VERSION/string_features

61it [21:48, 21.46s/it]


In [7]:
def pre_pseudo_gt_feature_generation(string_features_path, context_path, output_path):
    file_list = glob.glob(string_features_path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        if os.path.getsize(file) == 0:
            continue
        filename = file.split('/')[-1]        
        context_file = f"{context_path}/{filename[:-4]}_context.tsv"
        output_file = f"{output_path}/{filename}"
        !tl --log-file $tl_log_file pick-hc-candidates -o ignore_candidate $file \
        -s monge_elkan,monge_elkan_aliases,jaro_winkler,levenshtein \
        / context-match --debug --context-file $context_file --ignore-column-name ignore_candidate -o context_score \
        / kth-percentile -c context_score -o kth_percenter --ignore-column ignore_candidate --k-percentile 0.75  --minimum-cells 10 > $output_file

In [8]:
pre_pseudo_gt_feature_generation(train_string_feature_path,train_context_path, train_context_feature_path)

39it [10:51,  8.58s/it]

/Users/amandeep/Github/table-linker/tl/cli/kth-percentile.py:44: DtypeWarning: Columns (25,26,27,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  kp = KthPercentile(input_file=kwargs['input_file'],


104it [25:15, 10.88s/it]

/Users/amandeep/Github/table-linker/tl/cli/kth-percentile.py:44: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  kp = KthPercentile(input_file=kwargs['input_file'],


124it [28:32,  7.87s/it]

...

330it [1:12:09,  8.83s/it]

/Users/amandeep/Github/table-linker/tl/cli/kth-percentile.py:44: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  kp = KthPercentile(input_file=kwargs['input_file'],


334it [1:12:38,  7.40s/it]

/Users/amandeep/Github/table-linker/tl/cli/kth-percentile.py:44: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  kp = KthPercentile(input_file=kwargs['input_file'],


335it [1:12:53, 13.05s/it]


In [9]:
pre_pseudo_gt_feature_generation(dev_string_feature_path,dev_context_path, dev_context_feature_path)

56it [11:18,  8.27s/it]

/Users/amandeep/Github/table-linker/tl/cli/kth-percentile.py:44: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  kp = KthPercentile(input_file=kwargs['input_file'],


57it [11:31,  9.63s/it]

/Users/amandeep/Github/table-linker/tl/cli/kth-percentile.py:44: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  kp = KthPercentile(input_file=kwargs['input_file'],


58it [11:37,  8.63s/it]

/Users/amandeep/Github/table-linker/tl/cli/kth-percentile.py:44: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  kp = KthPercentile(input_file=kwargs['input_file'],


61it [11:59, 11.79s/it]


In [10]:
def rest_of_features_generation(context_features_path, class_count_dir, property_count_dir, output_path):
    file_list = glob.glob(context_features_path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        filename = file.split('/')[-1]
        if os.path.getsize(file) == 0:
            continue
        class_count_file = f"{class_count_dir}/{filename[:-4]}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename[:-4]}_prop_count.tsv"
        output_file = f"{output_path}/{filename}"
        !tl  --log-file $tl_log_file pgt-semantic-tf-idf $file \
            -o smc_class_score \
            --pagerank-column pagerank \
            --retrieval-score-column retrieval_score \
            --feature-file "$class_count_file" \
            --feature-name class_count \
            --high-confidence-column kth_percenter \
            / pgt-semantic-tf-idf \
            -o smc_property_score \
            --pagerank-column pagerank \
            --retrieval-score-column retrieval_score \
            --feature-file "$property_count_file" \
            --feature-name property_count \
            --high-confidence-column kth_percenter \
            > $output_file

In [11]:
rest_of_features_generation(train_context_feature_path, train_class_count, train_prop_count, train_feature_path)

39it [03:00,  4.83s/it]

/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],
/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,30,31,32,36) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],


104it [08:04,  4.94s/it]

/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],
/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,36) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],


124it [09:29,  4.27s/it]

...

330it [26:18,  5.55s/it]

/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],
/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,36) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],


334it [26:37,  4.66s/it]

/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],
/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,36) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],


335it [26:44,  4.79s/it]


In [12]:
rest_of_features_generation(dev_context_feature_path, dev_class_count, dev_prop_count, dev_feature_path)

56it [04:17,  5.16s/it]

/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],
/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,36) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],


57it [04:22,  5.22s/it]

/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],
/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,36) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],


58it [04:27,  5.27s/it]

/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],
/Users/amandeep/Github/table-linker/tl/cli/pgt-semantic-tf-idf.py:46: DtypeWarning: Columns (25,26,27,36) have mixed types.Specify dtype option on import or set low_memory=False.
  tfidf_unit = SemanticsFeature(kwargs['output_column_name'],


61it [04:43,  4.64s/it]


### Generate Training Data

In [ ]:
def merge_files(args):
    datapath = args.train_path
    df_list  = []
    for fn in glob.glob(f"{datapath}/*csv"):
        if os.path.getsize(fn) == 0:
            continue
        fid = fn.split('/')[-1][:-4]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df['context_score'].fillna(0.0, inplace=True)
        df_list.append(df)            
    return pd.concat(df_list) 

def compute_normalization_factor(args, all_data):
    min_max_scaler_path = args.min_max_scaler_path
    all_data_features = all_data[features]
    scaler = MinMaxScaler()
    scaler.fit(all_data_features)
    pickle.dump(scaler, open(min_max_scaler_path, 'wb'))
    return scaler

In [ ]:
def generate_train_data(args):
    scaler_path = args.min_max_scaler_path
    scaler = pickle.load(open(scaler_path, 'rb'))
    final_list = []
    sfeatures = copy.deepcopy(features) + ['evaluation_label']
    normalize_features = features
    evaluation_label = ['evaluation_label']
    positive_features_final = []
    negative_features_final = []
    for i,file in enumerate(glob.glob(args.train_path + '/*.csv')):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
                continue
        d_sample = pd.read_csv(file)
        d_sample['context_score'].fillna(0.0, inplace=True)
        grouped_obj = d_sample.groupby(['column', 'row'])
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            pos_features = []
            neg_features = []
            a = cell[1][cell[1]['evaluation_label'] == 1]
            if a.empty:
                continue
            pos_rows = cell[1][(cell[1]['evaluation_label'].astype(int) == 1) & (cell[1]['ignore_candidate'].astype(int) == 0)][features].to_numpy()
            for i in range(len(pos_rows)):
                pos_features.append(pos_rows[i])
            neg_rows = cell[1][(cell[1]['evaluation_label'].astype(int) == -1) & (cell[1]['ignore_candidate'].astype(int) == 0)][features].to_numpy()
            for i in range(min(50,len(neg_rows))):
                neg_features.append(neg_rows[i])
            random.shuffle(pos_features)
            random.shuffle(neg_features)
            positive_features_final.append(pos_features)
            negative_features_final.append(neg_features)
            
    print(len(positive_features_final), len(positive_features_final[3]))
    print(len(negative_features_final), len(negative_features_final[3]))
    pickle.dump(positive_features_final,open(args.pos_output,'wb'))
    pickle.dump(negative_features_final,open(args.neg_output,'wb'))

In [ ]:
gen_training_data_args = Namespace(train_path=train_feature_path, pos_output=pos_output, neg_output=neg_output, 
                 min_max_scaler_path=min_max_scaler_path)
all_data = merge_files(gen_training_data_args)
scaler = compute_normalization_factor(gen_training_data_args, all_data)

In [ ]:
len(all_data)

In [ ]:
all_data.head()

In [ ]:
pos = all_data[(all_data['evaluation_label'] == 1) & (all_data['ignore_candidate'] == 0)]
neg = all_data[(all_data['evaluation_label'] == -1) & (all_data['ignore_candidate'] == 0)]
print(len(pos))
print(len(neg))
pos.head()

In [ ]:
generate_train_data(gen_training_data_args)

### Model Definition

In [ ]:
# Dataset
class T2DV2Dataset(Dataset):
    def __init__(self, pos_features, neg_features):
        self.pos_features = pos_features
        self.neg_features = neg_features
    
    def __len__(self):
        return len(self.pos_features)
    
    def __getitem__(self, idx):
        return self.pos_features[idx], self.neg_features[idx]

# Model
class PairwiseNetwork(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        #original 12x24, 24x12, 12x12, 12x1
        self.fc1 = nn.Linear(hidden_size, 2*hidden_size)
        self.fc2 = nn.Linear(2*hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
    
    def forward(self, pos_features, neg_features):
        # Positive pass
        x = F.relu(self.fc1(pos_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pos_out = torch.sigmoid(self.fc4(x))
        
        # Negative Pass
        x = F.relu(self.fc1(neg_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        neg_out = torch.sigmoid(self.fc4(x))
        
        return pos_out, neg_out
    
    def predict(self, test_feat):
        x = F.relu(self.fc1(test_feat))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        test_out = torch.sigmoid(self.fc4(x))
        return test_out

# Pairwise Loss
class PairwiseLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.m = 0
    
    def forward(self, pos_out, neg_out):
        distance = (1 - pos_out) + neg_out
        loss = torch.mean(torch.max(torch.tensor(0), distance))
        return loss

### Training

In [ ]:
def generate_dataloader(positive_feat_path, negative_feat_path):
    pos_features = pickle.load(open(positive_feat_path, 'rb'))
    neg_features = pickle.load(open(negative_feat_path, 'rb'))

    pos_features_flatten = list(chain.from_iterable(pos_features))
    neg_features_flatten = list(chain.from_iterable(neg_features))

    train_dataset = T2DV2Dataset(pos_features_flatten, neg_features_flatten)
    train_dataloader = DataLoader(train_dataset, batch_size=64)
    return train_dataloader

def infer_scores(min_max_scaler_path, input_table_path, output_table_path, model):
    scaler = pickle.load(open(min_max_scaler_path, 'rb'))
    normalize_features = features
    for file in glob.glob(input_table_path + '/*.csv'):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
                continue
                
        d_sample = pd.read_csv(file)
        d_sample['context_score'].fillna(0.0, inplace=True)
        grouped_obj = d_sample.groupby(['column', 'row'])
        new_df_list = []
        pred = []
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            sorted_df = cell[1].sort_values('context_score', ascending=False)
            sorted_df_features = sorted_df[normalize_features]
            new_df_list.append(sorted_df)
            arr = sorted_df_features.to_numpy()
            test_inp = []
            for a in arr:
                test_inp.append(a)
            test_tensor = torch.tensor(test_inp).float()
            scores = model.predict(test_tensor)
            scores_list = torch.squeeze(scores).tolist()
            if not type(scores_list) is list:
                pred.append(scores_list)
            else:
                pred.extend(scores_list)
        test_df = pd.concat(new_df_list)
        test_df[final_score_column] = pred
        test_df.to_csv(f"{output_table_path}/{file_name}", index=False)

def train(args):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    
    else:
        device = torch.device('cpu')
    train_dataloader = generate_dataloader(args.positive_feat_path, args.negative_feat_path)
    criterion = PairwiseLoss()
    EPOCHS = args.num_epochs
    model = PairwiseNetwork(len(features)).to(device=device)
    optimizer = Adam(model.parameters(), lr=args.lr)
    top1_max_prec = 0
    for epoch in range(EPOCHS):
        train_epoch_loss = 0
        avg_loss = 0
        model.train()
        for bid, batch in tqdm(enumerate(train_dataloader), position=0, leave=True):
            positive_feat = torch.tensor(batch[0].float())
            negative_feat = torch.tensor(batch[1].float())
            optimizer.zero_grad()
            pos_out, neg_out = model(positive_feat, negative_feat)
            loss = criterion(pos_out, neg_out)
            loss.backward()
            optimizer.step()
            train_epoch_loss += loss
        avg_loss = train_epoch_loss / bid

        # Evaluation
        model.eval()
        infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
        eval_data = merge_eval_files(args.dev_output)
        res, candidate_eval_data = parse_eval_files_stats(eval_data, final_score_column)
        top1_precision = res['num_tasks_with_model_score_top_one_accurate']/res['num_tasks_with_gt']
        if top1_precision > top1_max_prec:
            top1_max_prec = top1_precision
            model_save_name = 'epoch_{}_loss_{}_top1_{}.pth'.format(epoch, avg_loss, top1_max_prec)
            best_model_path = os.path.join(args.model_save_path, model_save_name)
            torch.save(model.state_dict(), best_model_path)
        
        print("Epoch {}, Avg Loss is {}, epoch top1 {}, max top1 {}".format(epoch, avg_loss, top1_precision, top1_max_prec))
    return best_model_path

In [ ]:
def merge_eval_files(final_score_path):
    eval_file_names = []
    df_list = []
    for (dirpath, dirnames, filenames) in os.walk(final_score_path):
        for fn in filenames:
            if "csv" not in fn:
                continue
            abs_fn = os.path.join(dirpath, fn)
            assert os.path.isfile(abs_fn)
            if os.path.getsize(abs_fn) == 0:
                continue
            eval_file_names.append(abs_fn)
    
    for fn in eval_file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df_list.append(df)
    return pd.concat(df_list)

def parse_eval_files_stats(eval_data, method):
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'column', 'row'])['table_id'].count().reset_index(name="count")
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'column', 'row']))
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    has_gt_list = []
    has_gt_in_candidate = []
    for i, row in candidate_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
                    
        #rank on model score
        s_data = c_e_data.sort_values(by=[method], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)
            
    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    return res, candidate_eval_data

In [ ]:
training_args = Namespace(num_epochs=20, lr=0.001, positive_feat_path=pos_output, negative_feat_path=neg_output,
                         dev_path=dev_feature_path, dev_output=dev_output_predictions,
                         model_save_path=model_save_path, min_max_scaler_path=min_max_scaler_path)

In [ ]:
## Call Training
best_model_path = train(training_args)

In [ ]:
best_model_path

In [ ]:
best_model_path = '/Users/amandeep/Github/table-linker/data/SemTab2020/Round4/table-linker/dev-output/v17/saved_models/epoch_6_loss_0.18199089169502258_top1_0.7363788767812238.pth'

## Dev Prediction

In [ ]:
def dev_prediction(dev_feature_path, dev_predictions_top_k, saved_model, output_column, min_max_scaler_path, k=5):
    for file in glob.glob(dev_feature_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        feature_str =  ",".join(features)
        if os.path.getsize(file) == 0:
            continue
        # location where the output generated by the predictions wil be stored.
        dev_output = f"{dev_predictions_top_k}/{filename}"
        !tl --log-file $tl_log_file predict-using-model $file -o $output_column \
            --features $feature_str \
            --ranking-model $saved_model \
            --normalization-factor $min_max_scaler_path \
            --ignore-column ignore_candidate \
            / create-pseudo-gt \
            --column-thresholds $threshold \
            --filter smc_class_score:0 \
            / get-kg-links -c pseudo_gt -k $k --k-rows \
            > $dev_output

In [ ]:
def add_color(dev_predictions_top_k, dev_colorized_path, score_column, k=5):
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
                
        dev_color_file = f"{dev_colorized_path}/{filename[:-4]}.xlsx"
        !tl add-color $file -c "$score_column,evaluation_label" -k $k --output $dev_color_file

In [ ]:
def compute_metrics(dev_predictions_top_k, dev_predictions_metrics, score_column, k=5):
    df_list = []
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
                
        dev_metrics_file = f"{dev_predictions_metrics}/{filename}"
        !tl --log-file $tl_log_file metrics $file -k $k -c $score_column --tag $filename> $dev_metrics_file
        df_list.append(pd.read_csv(dev_metrics_file))
    return pd.concat(df_list)

In [ ]:
def compute_custom_metrics(dev_predictions_top_k):
    df_list = []
    for file in glob.glob(dev_predictions_top_k+"/*.csv"):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        col_df = []
        for col, coldf in df.groupby(by=["column"]):
            rows = 0
            pgt_rows = 0
            pgt_recall = 0
            unignored_rows = 0
            unignored_candidates = 0
            unignored_correct = 0
            ignored_correct = 0
            kth_perc_rows = 0
            kth_perc_correct = 0
            kth_perc_candidates = 0
            for row, rowdf in coldf.groupby(by=["row"]):
                rows += 1
                p_count = rowdf[(rowdf["pseudo_gt"] == 1)].shape[0]
                if p_count > 0:
                    pgt_rows += 1
                p_recall = rowdf[((rowdf["pseudo_gt"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
                pgt_recall += p_recall
                ignore_0_count = rowdf[rowdf["ignore_candidate"] == 0].shape[0]
                if ignore_0_count > 0:
                    unignored_rows += 1
                unignored_candidates += ignore_0_count
                unignored_correct += rowdf[((rowdf["ignore_candidate"] == 0) & (rowdf["evaluation_label"] == 1))].shape[0]
                ignored_correct += rowdf[((rowdf["ignore_candidate"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
                kth_perc_1_count = rowdf[rowdf["kth_percenter"] == 1].shape[0]
                if kth_perc_1_count > 0:
                    kth_perc_rows += 1
                kth_perc_candidates += kth_perc_1_count
                kth_perc_correct += rowdf[((rowdf["kth_percenter"] == 1) & (rowdf["evaluation_label"] == 1))].shape[0]
            unique_rows = coldf["label"].nunique()
            col_df.append(pd.DataFrame([{
                "filename":filename,
                "column": col,
                "rows": rows,
                "unique_rows": unique_rows,
                "pgt_rows": pgt_rows,
                "pgt_recall": pgt_recall,
                "pgt_accuracy": pgt_recall/pgt_rows if pgt_rows!=0 else 0,
                "unignored_rows": unignored_rows,
                "unignored_candidates": unignored_candidates,
                "unignored_correct": unignored_correct,
                "ignored_correct": ignored_correct,
                "ignore_candidate_accuracy": unignored_correct/unignored_rows if unignored_rows != 0 else 0,
                "kth_percenter_rows": kth_perc_rows,
                "kth_percenter_candidates": kth_perc_candidates,
                "kth_percenter_correct": kth_perc_correct,
                "kth_percenter_accuracy": kth_perc_correct/kth_perc_rows
            }]))
        df_list.append(pd.concat(col_df))
    return pd.concat(df_list)

In [ ]:
print(dev_feature_path, dev_predictions_top_k, best_model_path, final_score_column, min_max_scaler_path)

In [ ]:
dev_prediction(dev_feature_path, dev_predictions_top_k, best_model_path, final_score_column, min_max_scaler_path, k=5)

In [ ]:
metrics_df = compute_custom_metrics(dev_predictions_top_k)

In [ ]:
metrics_df

In [ ]:
metrics_df.to_csv(f"{dev_metrics_path}/metrics_1.csv", index=False)

In [ ]:
add_color(dev_predictions_top_k, dev_colorized_path, final_score_column)

In [ ]:
def find_missing_correct_candidates(candidates_path, missing_correct_candidates_path):
     for file in tqdm(glob.glob(candidates_path + '/*.csv')):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
        missing_file = f"{missing_correct_candidates_path}/{filename}"
        !tl check-candidates "$file" > "$missing_file"

In [ ]:
find_missing_correct_candidates(dev_candidate_path, dev_missing_candidates_path)

In [ ]:
def concat_files(files_path):
    df_list = []
    for file in glob.glob(files_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        df['filename'] = filename
        df_list.append(df)
    return pd.concat(df_list)

In [ ]:
missing_df = concat_files(dev_missing_candidates_path)

In [ ]:
missing_df

In [ ]:
missing_df.to_csv(f"{dev_missing_candidates_path}/missing_concatenated.csv", index=False)

In [ ]:
find_missing_correct_candidates(train_candidates_path, train_missing_candidates_path)

In [ ]:
train_missing_df = concat_files(train_missing_candidates_path)

In [ ]:
len(train_missing_df)

In [ ]:
train_missing_df

In [ ]:
train_missing_df.to_csv(f"{train_missing_candidates_path}/missing_concatenated.csv", index=False)